# Identifiability Example
# In this notebook, we will demonstrate how to use the `find_confounded_paths` function to identify confounded paths in a directed acyclic graph (DAG) and check the identifiability of queries using the `check_identifiable_query` function.

Algorithms 5 and 6 in DeCaFlow paper.

In [1]:
import pytest
import torch
from decaflow.utils.identifiability import (find_confounded_paths,
                                            check_identifiable_query,
                                            check_identifiable_query_on_all_descendants)
import networkx as nx

In [2]:
miao_graph = nx.DiGraph()

miao_graph.add_edges_from([
    ('Z', 'W'),
    ('Z', 'T'),
    ('Z', 'N'),
    ('Z', 'Y'),
    ('W', 'Y'),
    ('N', 'T'),
    ('T', 'Y')
])
non_confounded_set, confounded_dict, frontdoor_set  = find_confounded_paths(miao_graph, ['Z'], frontdoor=True)
queries = [('N', 'T'), ('W', 'Y'), ('T', 'Y')]
assert ('N', 'T') in confounded_dict.keys()
assert ('W', 'Y') in confounded_dict.keys()
assert('T', 'Y') in confounded_dict.keys()

assert check_identifiable_query(miao_graph, ('T', 'Y'), confounded_dict, ['Z'])

assert not check_identifiable_query(miao_graph, ('N', 'T'), confounded_dict, ['Z'])
assert not check_identifiable_query(miao_graph, ('W', 'Y'), confounded_dict, ['Z'])

for query in queries:
    if query in non_confounded_set:
        print(f'Query {query} is non-confounded')
    else:
        is_id = check_identifiable_query(miao_graph, query, confounded_dict, ['Z'])
        if is_id:
            print(f'Query {query} is identifiable')
        else:
            print(f'Query {query} is not identifiable')

Query ('N', 'T') is not identifiable
Query ('W', 'Y') is not identifiable
Query ('T', 'Y') is identifiable


In [3]:
two_outcomes_graph = nx.DiGraph()
two_outcomes_graph.add_edges_from([
    ('Z', 'T'),
    ('Z', 'Y1'),
    ('T', 'Y1'),
    ('T', 'Y2')
])

non_confounded_set, confounded_dict, frontdoor_set  = find_confounded_paths(two_outcomes_graph, ['Z'], frontdoor=True)
is_id = check_identifiable_query_on_all_descendants(two_outcomes_graph, 'T', confounded_dict, ['Z'])
print('T is identifiable on all descendants:', is_id)

T is identifiable on all descendants: False
